#Z-score + 공통 학교 필터링 템플릿
Z-score 방식으로 이상치를 제거하고, 모든 연도에 공통으로 존재하는 학교만 필터링

In [ ]:
import pandas as pd
from scipy.stats import zscore

# 파일 업로드
from google.colab import files
uploaded = files.upload()

# 데이터 읽기
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)
df.head()


In [ ]:
# 사용할 피처 목록 정의
feature_cols = [
    '중도탈락율', '기숙사 수용률', '전임교원 1인당 학생수(정원기준)', '전임교원 1인당 학생수(재학생기준)',
    '졸업자 수', '진학률', '정원내 신입생 충원율(%)', '기숙사 경쟁률', '가중 평균 재학생 충원율(%)',
    '총 강좌 수', '소규모 강좌 비율', '평균 강좌 규모 점수', '1인당장학금', '취업률'
]

# 결측치 제거
df_valid = df.dropna(subset=feature_cols)
print("결측치 제거 후 행 수:", len(df_valid))


In [ ]:
# Z-score 이상치 제거 함수
def remove_outliers_zscore(df, columns, threshold=3.0):
    df_z = df.copy()
    z_scores = df_z[columns].apply(zscore)
    mask = (z_scores.abs() < threshold).all(axis=1)
    return df_z[mask]

# 이상치 제거 실행
df_cleaned = remove_outliers_zscore(df_valid, feature_cols, threshold=3.0)
print("이상치 제거 후 행 수:", len(df_cleaned))


In [ ]:
# 모든 연도에 공통으로 존재하는 학교 추출
schools_per_year = df_cleaned.groupby("기준년도")["학교명"].apply(set)
common_schools = set.intersection(*schools_per_year)

# 공통 학교만 필터링
df_common = df_cleaned[df_cleaned["학교명"].isin(common_schools)]
print("공통 학교 수:", len(common_schools))
df_common.head()


In [ ]:
# 결과 저장 (UTF-8 with BOM 인코딩으로 Excel 호환)
df_common.to_csv("Zscore_공통학교만_이상치제거_통합데이터_UTF8BOM.csv", index=False, encoding="utf-8-sig")
files.download("Zscore_공통학교만_이상치제거_통합데이터_UTF8BOM.csv")
